In [26]:
S = """V PARKU KULTURY I ODDYHA

V parku kultury i oddyha je napisano:
"Tovariši! Ležati na zemji v parku kultury i oddyha ne jest kulturno.
Direktor parka kultury i oddyha"

Vladimir leži na zemji v parku kultury i oddyha. Začto on leži na zemji? Či ne znaje li on, že ležęt na zemji jedino nekulturni, necivilizovani ljudi? On vědaje tuto, ale mu vse jest jedno. On ljubi ležati na zemji i myzljiti o ziťju.

Direktor parka:
-- Hlopče, čemu ty ležiš na zemji? Ty jesi imal znati, že my imamo tut lavky. Tovariš Borodin sidi na lavkě. Ja sidžų na lavkě. Vsi, s izključeńjem tebe, sidęt na lavkah.
[pauza, poka Vladimir myslji]
Vladimir: Mně vse jest jedno. Ja ne mogų siděti.
Direktor: Začto ty ne možeš siděti?
Vladimir: Ja ne možem siděti, bo jesm slaby. Jesm slab, tovarišu direktore, jesm slaby v rezultatu glåda.
Direktor: Či vědaje li tvoj otec, že jesi slaby vslěd glåda?
Vladimir: Otec vědaje o tom, no ale jemu vse jest jedno. Ves den on leži v domu i pije alkohol.
Direktor: [s omŕžeńjem] Kaky žestoky člověk!
Vladimir: [sȯ skrbju] Tovarišu direktore, nikto mę ne ljubi. Nikto ne råzuměje mene. Ja jesm sam. Ja jesm jedin.

Direktor plače.
"""

In [6]:
import sys
sys.path.append("C://dev/ISV_data_gathering/")
sys.path.append("C://dev/")

from translation_aux import prepare_slovnik
from isv_translate import get_slovnik, download_slovnik

In [7]:
from isv_nlp_utils.slovnik import get_slovnik

In [11]:
from isv_nlp_utils import constants

In [8]:
dfs = get_slovnik()
slovnik = dfs['words']
prepare_slovnik(slovnik)


Found 'slovnik.pkl' file, using it


In [10]:
from razdel import tokenize



In [12]:
etm_morph = constants.create_analyzers_for_every_alphabet(r"C:\dev\ISV_data_gathering\\")['etm']
isv_dict = etm_morph._units[0][0].dict

In [14]:
from collections import Counter

def extract_stem_prefix_suffix(word, word_parse, isv_dict, postprocess_freq_thres=1.0):
    
    paradigm_num = word_parse[4][0][2]
    form_num = word_parse[4][0][3]
    paradigm = isv_dict.build_paradigm_info(paradigm_num)
    stem = isv_dict.build_stem(isv_dict.paradigms[paradigm_num], form_num, word)

    pref = paradigm[form_num][0]
    suff = paradigm[form_num][2]

    fixed_stem = stem
    fixed_suf = suff

    for pref_len in range(1, 5):
        freq = Counter([suf[:pref_len] for (pref, tags, suf) in paradigm])
        most_common_pref, its_freq = freq.most_common(1)[0]
        if its_freq / len(paradigm) >= postprocess_freq_thres:
            fixed_stem = stem + most_common_pref
            fixed_suf = suff[pref_len:]
            # print(stem, suff, "->", fixed_stem, fixed_suf)
    
    return pref, fixed_stem, fixed_suf


In [296]:
def join_morphemes(yaml_str):
    return ''.join([list(m.keys())[0] for m in yaml_str])

In [27]:
def add_whitespace_after_tokens(S, token_list):
    result = []
    for idx, token in enumerate(token_list):
        #result.append(token.text)
        if idx < len(token_list) - 1:
            next_token_start = token_list[idx + 1].start
            current_token_stop = token.stop
            whitespace_length = next_token_start - current_token_stop
            if whitespace_length:
                result.append(S[current_token_stop: next_token_start])
            else:
                result.append('')
        else:
            result.append('')
    return result

tokenized_S = list(tokenize(S))
whitespace = add_whitespace_after_tokens(S, tokenized_S)

In [28]:
len(whitespace), len(tokenized_S)

(258, 258)

In [29]:
res = """paragrafoj:
  -"""
yaml_pref = "        "
yaml_sep = """
    - token:
"""

all_entries2 = []

#print(res)
for token, token_whitespace in zip(tokenized_S, whitespace):
    # print(token.text, etm_morph.word_is_known(token.text))
    word = token.text
    if word.isalpha() and etm_morph.parse(word):
        word_parse = etm_morph.parse(word)[0]
        _, st, su = extract_stem_prefix_suffix(word, word_parse, isv_dict, 0.9)
        #morphemes = {st: 'stem', su: str(word_parse.tag).partition(" ")[2]}
        morphemes = {st: 'stem', su: str(word_parse.tag).replace(",", " ")}
        if "" in morphemes: del morphemes[""]
        entry = {"lemma": word_parse.normal_form.replace("dʒ", "đ"), "morfemes": morphemes}
        all_entries2.append(entry)
    else:
        all_entries2.append([yaml_sep.strip("\n") + " " + f"'{word}'"])
    if token_whitespace != "":
        if "\n" not in  token_whitespace:
            all_entries2.append(["    - " + token_whitespace])
        else:
            all_entries2.append(["  - "])

        #print([f"'{word}'"])

#print(
#    res + yaml_sep.join(yaml_pref + subentry
#        for entry in all_entries2
#        for subentry in entry
#    )
#)

In [31]:
print(res)
for entry in all_entries2:
    #print(entry)
    if type(entry) is dict:
        print(yaml_sep.strip("\n"))
        print(yaml_pref + "lemma: " + entry['lemma'])
        # if "morfemes" in entry:
        print(yaml_pref + "morfemes:")
        # print("  " + yaml_pref + "morfemes: " + entry['morfemes'])
        for mrp in entry["morfemes"].items():
            print(yaml_pref + " - " + mrp[0] + ": " + mrp[1])
        # "".join(yaml_pref + subentry)
        #for entry in all_entries2
        #for subentry in entry
    else:
        #if entry[0] not in ("'.'", "','", "':'"):
        #    print(yaml_sep.strip("\n"))

        #print(yaml_pref[2:] + str(entry[0]))
        print(str(entry[0]))
        


paragrafoj:
  -
    - token:
        lemma: v
        morfemes:
         - V: stem
    -  
    - token:
        lemma: park
        morfemes:
         - PARk: stem
         - u: masc NOUN sing loct
    -  
    - token:
        lemma: kultura
        morfemes:
         - KULTUR: stem
         - y: femn NOUN nomn plur
    -  
    - token:
        lemma: i
        morfemes:
         - I: stem
    -  
    - token:
        lemma: oddyh
        morfemes:
         - ODDYh: stem
         - a: masc NOUN sing gent
  - 
    - token:
        lemma: v
        morfemes:
         - V: stem
    -  
    - token:
        lemma: park
        morfemes:
         - park: stem
         - u: masc NOUN sing loct
    -  
    - token:
        lemma: kultura
        morfemes:
         - kultur: stem
         - y: femn NOUN nomn plur
    -  
    - token:
        lemma: i
        morfemes:
         - i: stem
    -  
    - token:
        lemma: oddyh
        morfemes:
         - oddyh: stem
         - a: masc NOUN s

In [3]:

with open(f"C:/dev/kurso-zagreba-metodo/html_generiloj/output/assets/css/main.css", "r", encoding="utf8") as f:
    main_css = "<style>" + f.read() + "</style>"
with open(f"C:/dev/kurso-zagreba-metodo/html_generiloj/output/assets/css/vortaro.css", "r", encoding="utf8") as f:
    vortaro_css = "<style>" + f.read() + "</style>"
with open(f"C:/dev/kurso-zagreba-metodo/html_generiloj/output/assets/js/main.js", "r", encoding="utf8") as f:
    main_js = "<script>" + f.read() + "</script>"
with open(f"C:/dev/kurso-zagreba-metodo/html_generiloj/output/assets/js/vortaro.js", "r", encoding="utf8") as f:
    vortaro_js =  "<script>" + f.read() + "</script>"
with open(f"C:/dev/kurso-zagreba-metodo/html_generiloj/output/en/js/vortlisto.js", "r", encoding="utf8") as f:
    vortlisto_js =  "<script>" + f.read() + "</script>"


In [4]:
REPLACEMENTS = {
         '    <link href="/en/../assets/css/main.css" rel="stylesheet">\n': main_css,
         '    <link href="/en/../assets/css/vortaro.css" rel="stylesheet">\n': vortaro_css,
         '    <script src="/en/../assets/js/main.js"></script>\n': main_js,
         '    <script src="/en/js/vortlisto.js"></script>\n': vortlisto_js,
         '    <script src="/en/../assets/js/vortaro.js"></script>\n': vortaro_js,
}

In [32]:
import glob

for filename in glob.glob(r"C:\dev\kurso-zagreba-metodo\html_generiloj\output\en\**\*.html", recursive=True):
    if "01" in filename:
        print(filename)
        new_content = []
        with open(filename, "r", encoding="utf8") as f:
            for line in f:
                if line in REPLACEMENTS:
                    new_content.append(REPLACEMENTS[line])
                else:
                    new_content.append(line)
        with open(filename, "w", encoding="utf8") as f:
            f.write("".join(new_content))


C:\dev\kurso-zagreba-metodo\html_generiloj\output\en\01\index.html
C:\dev\kurso-zagreba-metodo\html_generiloj\output\en\01\ekzerco1\index.html
C:\dev\kurso-zagreba-metodo\html_generiloj\output\en\01\ekzerco2\index.html
C:\dev\kurso-zagreba-metodo\html_generiloj\output\en\01\ekzerco3\index.html
C:\dev\kurso-zagreba-metodo\html_generiloj\output\en\01\gramatiko\index.html
C:\dev\kurso-zagreba-metodo\html_generiloj\output\en\01\vortoj\words.html
